# Ontologías
Otra manera fundamentalmente diferente de representar el lenguaje (y el conocimiento) son las ontologías. Ellas estbalecen no solo relaciones, sino tipos de relaciones entre palabras.
Vamos a explorar una de las ontologías más conocidas: Wordnet

In [2]:
import sklearn
import os, re, string
#import spacy
import nltk
import numpy, matplotlib
import pandas as pd
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.corpus import wordnet 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


##Los synsets
Es como tener un diccionario ampliado (tesauro) de términos, abordando de otro modo el conocimiento. Por ejemplo, ver todos los significados/sentidos/sinónimos de una palabra.

In [ ]:
wordnet.synsets("cat")

[Synset('cat.n.01'),
 Synset('guy.n.01'),
 Synset('cat.n.03'),
 Synset('kat.n.01'),
 Synset('cat-o'-nine-tails.n.01'),
 Synset('caterpillar.n.02'),
 Synset('big_cat.n.01'),
 Synset('computerized_tomography.n.01'),
 Synset('cat.v.01'),
 Synset('vomit.v.01')]

Es posible conectarse a la wordnet multilingue a través de la interfaz en NLTK. Por ejemplo, comparemos *house* en inglés con *casa* en español

In [4]:
wordnet.synsets("house")

[Synset('house.n.01'),
 Synset('firm.n.01'),
 Synset('house.n.03'),
 Synset('house.n.04'),
 Synset('house.n.05'),
 Synset('house.n.06'),
 Synset('house.n.07'),
 Synset('sign_of_the_zodiac.n.01'),
 Synset('house.n.09'),
 Synset('family.n.01'),
 Synset('theater.n.01'),
 Synset('house.n.12'),
 Synset('house.v.01'),
 Synset('house.v.02')]

In [ ]:
##extrae todos los posibles "significados"
wordnet.synsets("casa",lang="spa")


[Synset('diggings.n.02'),
 Synset('dwelling.n.01'),
 Synset('house.n.01'),
 Synset('house.n.06'),
 Synset('firm.n.01'),
 Synset('manufacturer.n.01'),
 Synset('family.n.01'),
 Synset('house.n.09'),
 Synset('home.n.03'),
 Synset('home.n.01'),
 Synset('home.n.07')]

Por tanto puede usarse como un traductor, yendo de un idioma al otro. Por supuesto, la ambiguedad del idioma y el contexto van a dificultar la tarea.

In [6]:
wordnet.synset("house.n.01").lemma_names("jpn")


['お宅',
 'お家',
 'ハウス',
 'メゾン',
 '人家',
 '人屋',
 '令堂',
 '住みか',
 '住み処',
 '住み家',
 '住宅',
 '住家',
 '住居',
 '住屋',
 '宅',
 '室家',
 '家',
 '家宅',
 '家屋',
 '宿',
 '居',
 '居宅',
 '居所',
 '居館',
 '屋',
 '屋宇',
 '建屋',
 '戸',
 '棲み家',
 '棲家',
 '館']

##Ambiguedad en la traducción
Al usarlo como el traductor de una palabra, obsérvese como crece la ambiguedad semántica. El problema de la traducción es claramente un problema de alta complejidad.

In [50]:
##puedo usarlo de algun modo como un traductor
wordnet.synsets("coucher",lang="fra")
francesingles=wordnet.synsets("coucher",lang="fra")
francesingles


[Synset('lay.v.02'), Synset('bed.n.02')]

Tuve que pasar por el inglés para ir al español

In [51]:
espanol=[]
for palabra in francesingles:
    espanol.append(" ".join(palabra.lemma_names("spa")))
espanol


['tumbar', 'lecho']

Y luego de vuelta al inglés y al francés

In [52]:
inglesback=[]
for pal in espanol:
    inglesback.append(wordnet.synsets(pal, lang="spa"))
inglesback


[[Synset('knock.v.01'),
  Synset('down.v.05'),
  Synset('poleax.v.01'),
  Synset('deck.v.03'),
  Synset('lay.v.02'),
  Synset('lie_down.v.01')],
 [Synset('bed.n.02'),
  Synset('bedding_material.n.01'),
  Synset('bed.n.03'),
  Synset('riverbed.n.01')]]

In [53]:
inglesfinal=[]
for element in inglesback:
    inglesfinal=inglesfinal+element
francesfinal=[]
for palabra in inglesfinal:
    francesfinal.append(palabra.lemma_names("fra"))
francesfinal


[[],
 ['abattre', 'démolir'],
 [],
 ['plancher', 'pont', 'étage'],
 ['coucher', 'déposer', 'poser'],
 ["s'étendre"],
 ['coucher', 'lit', 'parterre', 'planche'],
 ['litière'],
 ['lit'],
 []]

Trabajando con el *synset* más frecuente y con palabras menos ambiguas los resultados son mejores

In [60]:
synset_ingles=wordnet.synsets("lundi",lang="fra")
lema_español=synset_ingles[0].lemma_names("spa")
synset_español=wordnet.synsets(lema_español[0], lang="spa")
synset_español[0].lemma_names("fra")
##probar "nuit"

['Lundi', 'Monday', 'lundi', 'poniedziałek']

##Relaciones de los *synsets*
Parte importante de una ontología son las relaciones que pueden establecerse entre los elementos léxicos (para el caso de Wordnet, *synsets* o lemas de dichos *synsets*). Tenemos, para empezar, definiciones y ejemplos.


###Definiciones
para definir una palabra tenemos su definición de diccionario y ejemplos

In [64]:
inglesfinal[5].definition()


'assume a reclining position'

In [ ]:
inglesfinal[5].examples()


['lie down on the bed until you feel better']

###Antónimos
Una relación importante (usualmente de los adjetivos, pero puede ser también de verbos o incluso nombres) es los antónimos. Estos solo se pueden obtener de un lema específico. Recuerde que los sinónimos son los mismos *synsets*

In [62]:
inglesfinal[5].lemmas()

[Lemma('lie_down.v.01.lie_down'), Lemma('lie_down.v.01.lie')]

In [63]:
inglesfinal[5].lemmas()[0].antonyms()

[Lemma('arise.v.03.arise')]

###Taxonomías: Hiperonimia e hiponimia
El conocimiento está también clasificado en taxonomías, teniendo hiperónimos (categoría superior) e hipónimos (categoría inferior). Estas estructuras forman árboles taxonómicos de los cuales se pueden derivar diferentes relaciones y métricas.

In [65]:
dog = wordnet.synset('dog.n.01')
cat=wordnet.synset("cat.n.01")
duck=wordnet.synset("duck.n.01")
table=wordnet.synset("table.n.01")

dog.hypernyms()


[Synset('canine.n.02'), Synset('domestic_animal.n.01')]

In [66]:
cat.hypernyms()


[Synset('feline.n.01')]

In [67]:
dog.hyponyms()


[Synset('basenji.n.01'),
 Synset('corgi.n.01'),
 Synset('cur.n.01'),
 Synset('dalmatian.n.02'),
 Synset('great_pyrenees.n.01'),
 Synset('griffon.n.02'),
 Synset('hunting_dog.n.01'),
 Synset('lapdog.n.01'),
 Synset('leonberg.n.01'),
 Synset('mexican_hairless.n.01'),
 Synset('newfoundland.n.01'),
 Synset('pooch.n.01'),
 Synset('poodle.n.01'),
 Synset('pug.n.01'),
 Synset('puppy.n.01'),
 Synset('spitz.n.01'),
 Synset('toy_dog.n.01'),
 Synset('working_dog.n.01')]

In [68]:
dog.root_hypernyms()


[Synset('entity.n.01')]

In [69]:
dog.lemmas()


[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

###distancias taxonómicas
gracias a la estructura taxonómica podemos también relacionar palabras, y calcular sus distancias.

In [ ]:
wordnet.synset('dog.n.01').lowest_common_hypernyms(wordnet.synset('duck.n.01'))


[Synset('vertebrate.n.01')]

La distancia *path* tiene que ver con cuántos pasos tiene el camino taxonómico más corto hasta el otro concepto. Es básicamente el inverso del número de pasos requerido.

In [70]:
dog.path_similarity(table)


0.07142857142857142

In [ ]:
dog.path_similarity(cat)


0.2

In [72]:
dog.path_similarity(wordnet.synset('dalmatian.n.02'))

0.5

### Caso de uso con taxonomías
Como ejemplo, extraigamos de un listado de compras un elemento que sea un hipónimo

In [73]:
peticion="fruta"
productos=["lasaña","manzana","cerdo", "pera"]
frutas=wordnet.synsets(peticion,lang="spa")
frutas

[Synset('edible_fruit.n.01')]

In [74]:
todafruta=frutas[0].hyponyms()
todafruta

[Synset('ackee.n.01'),
 Synset('anchovy_pear.n.02'),
 Synset('apple.n.01'),
 Synset('apricot.n.02'),
 Synset('avocado.n.01'),
 Synset('banana.n.02'),
 Synset('barbados_gooseberry.n.02'),
 Synset('berry.n.01'),
 Synset('breadfruit.n.02'),
 Synset('canistel.n.02'),
 Synset('carambola.n.02'),
 Synset('carissa_plum.n.01'),
 Synset('ceriman.n.02'),
 Synset('cherry.n.03'),
 Synset('citrus.n.01'),
 Synset('cling.n.01'),
 Synset('cocoa_plum.n.02'),
 Synset('custard_apple.n.02'),
 Synset('date.n.08'),
 Synset('dried_fruit.n.01'),
 Synset('durian.n.02'),
 Synset('elderberry.n.02'),
 Synset('feijoa.n.02'),
 Synset('fig.n.04'),
 Synset('freestone.n.01'),
 Synset('garambulla.n.02'),
 Synset('genip.n.02'),
 Synset('genipap.n.01'),
 Synset('grape.n.01'),
 Synset('guava.n.03'),
 Synset('hog_plum.n.03'),
 Synset('hog_plum.n.04'),
 Synset('jaboticaba.n.02'),
 Synset('jackfruit.n.02'),
 Synset('jujube.n.02'),
 Synset('kai_apple.n.01'),
 Synset('ketembilla.n.02'),
 Synset('kiwi.n.03'),
 Synset('lanseh.n.0

In [86]:
fruta1=todafruta[2].lemma_names("spa")
fruta1

['manzana']

In [87]:
fruspa=[]
for palabra in todafruta:
    fruspa.append(" ".join(palabra.lemma_names("spa")))
fruspa

['',
 '',
 'manzana',
 '',
 'persea_americana',
 '',
 '',
 '',
 '',
 '',
 'averrhoa_carambola',
 '',
 '',
 '',
 'citrus',
 'variedad_Clingstone',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'higo',
 'variedad_Freestone',
 '',
 '',
 'huito jagua',
 'uva',
 '',
 '',
 '',
 '',
 '',
 'ziziphus_zizyphus',
 '',
 '',
 '',
 '',
 'litchi_chinensis',
 '',
 '',
 '',
 '',
 'mangostino',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'prunus_persica',
 'pera',
 '',
 '',
 '',
 '',
 'granada',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [88]:
for element in fruspa:
  if element in productos: print("Tenemos"+" " + element)

Tenemos manzana
Tenemos pera


## Otras relaciones: parte y todo
Las relaciones de parte y todo son llamadas *meronimia* (el merónimo es la parte) y *holonimia* (el holónimo es el todo). Hay relaciones de parte, de miembro y de sustancia.

Partes de una bicicleta

In [91]:
bike = wordnet.synset('bicycle.n.01')
bike.part_meronyms()

[Synset('bicycle_seat.n.01'),
 Synset('bicycle_wheel.n.01'),
 Synset('chain.n.03'),
 Synset('coaster_brake.n.01'),
 Synset('handlebar.n.01'),
 Synset('kickstand.n.01'),
 Synset('mudguard.n.01'),
 Synset('pedal.n.02'),
 Synset('sprocket.n.02')]

Busquemos el holónimo de una cabeza

In [93]:
wordnet.synsets("head")

[Synset('head.n.01'),
 Synset('head.n.02'),
 Synset('mind.n.01'),
 Synset('head.n.04'),
 Synset('head.n.05'),
 Synset('head.n.06'),
 Synset('head.n.07'),
 Synset('fountainhead.n.02'),
 Synset('head.n.09'),
 Synset('head.n.10'),
 Synset('head.n.11'),
 Synset('capitulum.n.01'),
 Synset('principal.n.02'),
 Synset('head.n.14'),
 Synset('head.n.15'),
 Synset('promontory.n.01'),
 Synset('head.n.17'),
 Synset('head.n.18'),
 Synset('forefront.n.01'),
 Synset('pass.n.09'),
 Synset('headway.n.02'),
 Synset('point.n.20'),
 Synset('question.n.02'),
 Synset('heading.n.01'),
 Synset('head.n.25'),
 Synset('head.n.26'),
 Synset('read/write_head.n.01'),
 Synset('head.n.28'),
 Synset('head.n.29'),
 Synset('head.n.30'),
 Synset('head.n.31'),
 Synset('drumhead.n.01'),
 Synset('oral_sex.n.01'),
 Synset('head.v.01'),
 Synset('head.v.02'),
 Synset('lead.v.04'),
 Synset('head.v.04'),
 Synset('steer.v.01'),
 Synset('head.v.06'),
 Synset('head.v.07'),
 Synset('head.v.08'),
 Synset('head.v.09')]

In [94]:
 wordnet.synset('head.n.01').part_holonyms()

[Synset('animal.n.01'), Synset('body.n.01')]

In [95]:
wordnet.synsets("man")

[Synset('man.n.01'),
 Synset('serviceman.n.01'),
 Synset('man.n.03'),
 Synset('homo.n.02'),
 Synset('man.n.05'),
 Synset('man.n.06'),
 Synset('valet.n.01'),
 Synset('man.n.08'),
 Synset('man.n.09'),
 Synset('man.n.10'),
 Synset('world.n.08'),
 Synset('man.v.01'),
 Synset('man.v.02')]

In [96]:
wordnet.synset('homo.n.02').part_meronyms()

[Synset('arm.n.01'),
 Synset('body_hair.n.01'),
 Synset('face.n.01'),
 Synset('foot.n.01'),
 Synset('hand.n.01'),
 Synset('human_body.n.01'),
 Synset('human_head.n.01'),
 Synset('loin.n.02'),
 Synset('mane.n.02')]

In [97]:
wordnet.synset('adult_male_body.n.01').part_meronyms()

[Synset('beard.n.01')]